In [ ]:
!apt-get install wget
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json ./storage/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json dev-v2.0.json

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.19.4-1ubuntu2.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
--2022-09-21 00:42:50--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json.1’

train-v2.0.json.1   100%[===================>]  40.17M  --.-KB/s    in 0.1s    

2022-09-21 00:42:50 (392 MB/s) - ‘train-v2.0.json.1’ saved [42123633/42123633]

--2022-09-21 00:42:50--  http://./storage/train-v2.0.json
Resolving . (.).

In [ ]:
# Make sure you have a GPU running
!nvidia-smi

Wed Sep 21 00:42:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Install the latest release of Haystack in your own environment
#! pip install farm-haystack

# Install the latest master of Haystack
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-53vw6m5g/farm-haystack_a0893bb4b5fc422caf9a33d39bf6bde1
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-53vw6m5g/farm-haystack_a0893bb4b5fc422caf9a33d39bf6bde1
  Resolved https://github.com/deepset-ai/haystack.git to commit 15bb6c2ea21057a1f44bc07084f8b852d7728048
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from haystack.nodes import FARMReader

In [ ]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)
data_dir = "./"
# data_dir = "PATH/TO_YOUR/TRAIN_DATA"

In [ ]:
reader.train(data_dir=data_dir, train_filename="dev-v2.0.json", use_gpu=True, n_epochs=4, save_dir="my_model",)

Train epoch 3/3 (Cur. train loss: 0.6755): 100%|██████████| 1360/1360 [05:22<00:00,  4.21it/s]


In [ ]:
# Saving the model happens automatically at the end of training into the `save_dir` you specified
# However, you could also save a reader manually again via:
reader.save(directory="/content/drive/MyDrive/my_model")

In [ ]:
# If you want to load it at a later point, just do:
new_reader = FARMReader(model_name_or_path="/content/drive/MyDrive/my_model")

In [ ]:
reader_eval_results = new_reader.eval_on_file("./", "dev-v2.0.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.
Evaluating: 100%|██████████| 272/272 [01:45<00:00,  2.59it/s]


In [ ]:
reader_eval_results

{'EM': 88.84247171453438,
 'f1': 89.45614522413196,
 'top_n_accuracy': 99.84334203655352,
 'top_n': 4,
 'EM_text_answer': 82.67984998295262,
 'f1_text_answer': 83.88188009295536,
 'top_n_accuracy_text_answer': 99.69314694851687,
 'top_n_EM_text_answer': 97.23832253665189,
 'top_n_f1_text_answer': 99.1530880418712,
 'Total_text_answer': 2933,
 'EM_no_answer': 95.27027027027027,
 'f1_no_answer': 95.27027027027027,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 2812}

In [ ]:
context = "Hi My name is Oluwadamifogore, some call me Engineer-D because I am a badass engineer, if you find it difficult to pronounce my name just call me David"

In [ ]:
new_reader.predict_on_texts("What is your name?",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 37.06 Batches/s]


{'query': 'What is your name?',
 'no_ans_gap': 3.1716909408569336,
 'answers': [<Answer {'answer': 'Oluwadamifogore', 'type': 'extractive', 'score': 0.6008038520812988, 'context': 'Hi My name is Oluwadamifogore, some call me Engineer-D because I am a badass engineer, if you find it difficult to pronounce my name just call me Davi', 'offsets_in_document': [{'start': 14, 'end': 29}], 'offsets_in_context': [{'start': 14, 'end': 29}], 'document_id': '6fe28b9d1e5275f19d85683611fd726d', 'meta': {}}>]}

In [ ]:
new_reader.predict_on_texts("What are you called?",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 48.00 Batches/s]


{'query': 'What are you called?',
 'no_ans_gap': -0.8229160308837891,
 'answers': [<Answer {'answer': 'David', 'type': 'extractive', 'score': 0.30875808000564575, 'context': 'i My name is Oluwadamifogore, some call me Engineer-D because I am a badass engineer, if you find it difficult to pronounce my name just call me David', 'offsets_in_document': [{'start': 146, 'end': 151}], 'offsets_in_context': [{'start': 145, 'end': 150}], 'document_id': '6fe28b9d1e5275f19d85683611fd726d', 'meta': {}}>]}

In [ ]:
new_reader.predict_on_texts("Who are you?",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 40.62 Batches/s]


{'query': 'Who are you?',
 'no_ans_gap': -0.3489704132080078,
 'answers': [<Answer {'answer': 'David', 'type': 'extractive', 'score': 0.4053061008453369, 'context': 'i My name is Oluwadamifogore, some call me Engineer-D because I am a badass engineer, if you find it difficult to pronounce my name just call me David', 'offsets_in_document': [{'start': 146, 'end': 151}], 'offsets_in_context': [{'start': 145, 'end': 150}], 'document_id': '6fe28b9d1e5275f19d85683611fd726d', 'meta': {}}>]}

In [ ]:
ans = new_reader.predict_on_texts("What is your name?",[context])

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00, 49.57 Batches/s]


In [ ]:
type(ans)

dict

In [ ]:
ans

{'query': 'What is your name?',
 'no_ans_gap': 3.1716909408569336,
 'answers': [<Answer {'answer': 'Oluwadamifogore', 'type': 'extractive', 'score': 0.6008038520812988, 'context': 'Hi My name is Oluwadamifogore, some call me Engineer-D because I am a badass engineer, if you find it difficult to pronounce my name just call me Davi', 'offsets_in_document': [{'start': 14, 'end': 29}], 'offsets_in_context': [{'start': 14, 'end': 29}], 'document_id': '6fe28b9d1e5275f19d85683611fd726d', 'meta': {}}>]}